# 8. Co-abundance analysis

This notebook is used for single-cell co-abundance analysis preparing the data for Figure 5.

Before starting, use 1.Preprocessing notebook to generate:
- SC2_normalized.h5ad - Normalized single-cell data
- SC3_normalized.h5ad - Normalized single-cell data

Download data available at Metabolights [www.ebi.ac.uk/metabolights/MTBLS11236], study identifier MTBLS11236 and move to data folder:
- SC2_metabolite_IDs.csv - Metabolite metadata
- SC3_metabolite_IDs.csv - Metabolite metadata

In [1]:
from pathlib import Path

import pandas as pd
import scanpy as sc
from scipy import stats
from anndata import read_h5ad

import sys
sys.path.insert(0, '../functions/')
import pl
import utils

### Set paths

In [2]:
#input
data_dir = Path(r'../data')
anndata_path1 = data_dir / 'SC2_normalized.h5ad'
metabolites_path1 = data_dir / 'SC2_metabolite_IDs.csv'

anndata_path2 = data_dir / 'SC3_normalized.h5ad'
metabolites_path2 = data_dir / 'SC3_metabolite_IDs.csv'

#output
data_path =  Path(r'../data')
data_path.mkdir(parents=True, exist_ok=True)
sc.settings.figdir = data_path

### Load data

In [3]:
#load anndata file
adata1 =  sc.read_h5ad(anndata_path1)
adata1.raw = adata1

adata2 =  sc.read_h5ad(anndata_path2)
adata2.raw = adata2

#load metabolites file
metabolite_ID1 = pd.read_csv(metabolites_path1)
metabolite_ID2 = pd.read_csv(metabolites_path2)

/Users/jeanydelafiori/Apps/miniconda3/envs/spacem_analysis/lib/python3.12/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


## Determine co-detected metabolites

Aiming to determinine if two metabolites are simultaneously detected in a single-cell for each cell line:
- Step 1: we atributted 0 or 1 for the absence or presence of non-zero ion intensity for each ion in a single-cell, respectively.
- Step 2: we combined ions two-by-two for every cell line ion intensity matrix
- Step 3: for each ion pair, we summarized the attributed values (0 or 1):
    - For cells where both ions have zero intensity, value = 0 (dropout)
    - For cells with one non-zero ion, value = 1 (mismatch)
    - For cells where both ions have non-zero intensity, value = 2 (co-detected)
- Step 4: Aiming a low dropout rate, we selected ion-pairs where the proportion of cells having 0 value were less or equal to 5%
- Step 5: Considering only these ion pairs, we filtered for ion-pairs where the proportion of cells having value 2 were more or equal to 60%

#### SC2

In [4]:
#calculate the number of cells having zero or non-zero intensity for an ion pair
cell_name= ['A498','BT-549','HOP-62','HS 578T','HT29','HeLa','IGR-OV1','MALME-3M','NCI-H460','OVCAR-5']

prop_list1 = []

for cell in cell_name: 
    adata_cell = adata1[adata1.obs.CellLine.eq(cell),:]
    prop_df = utils.calculate_cell_dropout(adata=adata_cell, cell=cell)

    #filter for ion pair with low droupout for both ions and co-detected in at least 60% of the cells
    filter_df = prop_df[(prop_df['proportion'] <= 0.05) & (prop_df['value'] == 0)]
    prop_df = prop_df[prop_df['ions'].isin(list(filter_df['ions']))]
    prop_df = prop_df[(prop_df['value']==2) & (prop_df['proportion']>=0.6)]

    prop_list1.append(prop_df)

detection_df1 = pd.concat(prop_list1)

#### SC3

In [5]:
#calculate the number of cells having zero or non-zero intensity for an ion pair
condition_name= ['Control', '2-DG 12h', '2-DG 24h']
obs = 'condition'

prop_list2 = []

for condition in condition_name: 
    adata_cond = adata2[adata2.obs[obs].eq(condition),:]
    prop_df = utils.calculate_cell_dropout(adata=adata_cond, cell=condition)

    #filter for ion pair with low droupout for both ions and co-detected in at least 60% of the cells
    filter_df = prop_df[(prop_df['proportion'] <= 0.05) & (prop_df['value'] == 0)]
    prop_df = prop_df[prop_df['ions'].isin(list(filter_df['ions']))]
    prop_df = prop_df[(prop_df['value']==2) & (prop_df['proportion']>=0.6)]

    prop_list2.append(prop_df)

detection_df2 = pd.concat(prop_list2)

## Coefficient of correlation

For every cell line ion intensity matrix we computed Pearson correlation coefficient among ion-pairs. Then, we retained only the ion pairs that passed the co-detection criteria.

#### SC2

In [5]:
#calculate correlation between ion pairs
cell_name = ['A498','BT-549','HOP-62','HS 578T','HT29','HeLa','IGR-OV1','MALME-3M','NCI-H460','OVCAR-5']
corr_cell = []

for cell in cell_name: 
    adata_cell = adata1[adata1.obs.CellLine.eq(cell),:]
    int_matrix = pd.DataFrame(columns=adata_cell.var_names, data = adata_cell.X, index = adata_cell.obs_names)
    corr_cell.extend(utils.ion_correlation_calculation(int_matrix=int_matrix, cell=cell))
    
corr_df1 = pd.DataFrame.from_dict(corr_cell)

/Users/jeanydelafiori/Documents/UCSD/HTSpaceM/notebooks/../functions/utils.py:249: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  res = stats.pearsonr(int_matrix[col_1], int_matrix[col_2], alternative='two-sided')


#### SC3

In [7]:
condition_name = ['Control', '2-DG 12h', '2-DG 24h']
obs = 'condition'
corr_condition = []

for condition in condition_name: 
    # Subset to the current condition
    adata_cond = adata2[adata2.obs[obs] == condition, :]

    # Loop over wells
    for well in adata_cond.obs['well'].unique():
        adata_well = adata_cond[adata_cond.obs['well'] == well, :]

        # Get expression matrix for that well
        int_matrix = pd.DataFrame(columns=adata_well.var_names, data=adata_well.X, index=adata_well.obs_names)

        # Calculate correlation within this well
        well_corrs = utils.ion_correlation_calculation(int_matrix=int_matrix, cell=condition)
        
        for c in well_corrs:
            c['well'] = well  # Tag with well
        corr_condition.extend(well_corrs)

# Turn into DataFrame
corr_df2 = pd.DataFrame.from_dict(corr_condition)

#Calculate mean correlation per condition
mean_corr = (corr_df2.groupby(['cell_line', 'ions'])[['corr', 'p-value']].mean().reset_index())
mean_corr[['col1', 'col2']] = mean_corr['ions'].str.split(',', expand=True)

## Determine co-abundant ions

We considered two ions co-abundant when they passed the co-detection criteria and presented a coefficient of correlation higher or equal to the absolute value 0.3
- Ion-pairs positively co-abundant have R higher or equal to +0.3
- Ion-pairs inversely co-abundant have R lower or equal to -0.3

In [6]:
def set_group(data):
    if data['corr'] >= 0.3:
        return 'positively co-abundant'
    elif data['corr'] <= -0.3:
        return 'negatively co-abundant'
    else:
        return 'co-detected'

#### SC2

In [7]:
coabundance_df1 = detection_df1.merge(corr_df1, how = 'inner')

coabundance_df1['group'] = coabundance_df1.apply(set_group, axis=1)
coabundance_df1.to_csv(data_path / 'SC2_co-abundance.csv', index=False)

#### SC3

In [10]:
coabundance_df2 = detection_df2.merge(mean_corr, how = 'inner')

coabundance_df2['group'] = coabundance_df2.apply(set_group, axis=1)
coabundance_df2.to_csv(data_path/ 'SC3_co-abundance.csv')